In [ ]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sys.path.append(os.path.abspath(".."))  
from src.graficos import criar_boxplot, criar_scatterplot_outliers, criar_barplot_stacked, criar_pieplot, criar_side_by_side, criar_histograma
from src.prints import imprimir_medias, imprimir_diferenca_medias
sns.set_theme()

In [ ]:
data = pd.read_csv("../data/cleaned.csv")

In [ ]:
data

Criação de lista "Métricas"

In [ ]:
metricas = ["pontuacao","regularidade", 
            "duracao", "sono_leve","sono_leve_perc", 
            "sono_profundo", "sono_profundo_perc", "REM", "REM_perc", 
            "tempo_acordado", "vezes_acordado"]

In [ ]:
criar_boxplot(dataframe=data, metricas=metricas)

In [ ]:
outliers = ["pontuacao", "sono_leve_perc", "REM", "REM_perc", "tempo_acordado"]

Visualização de outliers 

Utilização de Scater Plot para visualizar dados

Utilização de Line Plot para visualizar Linha Inferior, Linha Superior e Média

In [ ]:
criar_scatterplot_outliers(outliers=outliers, dataframe=data, nrows=2, ncols=3)

Visualização de Histogramas

In [ ]:
criar_histograma(dataframe=data, metricas=metricas, nrows=3, ncols=4)

In [ ]:
media_pontuacao = data["pontuacao"].mean()
media_pontuacao

Visualização geral de Pontuação e linha de média

In [ ]:
plt.figure(figsize=(20, 10))
sns.lineplot(data=data, x="data", y="pontuacao")
plt.axhline(media_pontuacao, color="red", linestyle="--")
plt.ylabel("Pontuação")
plt.xticks(rotation=90)
plt.xlabel("Data")
plt.show()

Gráfico de Correlação

In [ ]:
plt.figure(figsize=(16, 8))
sns.heatmap(data[metricas].corr(),  annot=True, cmap="coolwarm", center=0)
plt.title("Correlação das Métricas")
plt.show()

Utilização de Bar Plot (Stacked) para visualizar como está composta a duração do sono 

In [ ]:
composicao_sono = ["sono_leve", "sono_profundo", "REM"]

In [ ]:
criar_barplot_stacked(dataframe=data, composicao_sono=composicao_sono)

In [ ]:
total_sono = data[["sono_leve", "sono_profundo", "REM"]].sum()
labels = total_sono.index.tolist()
values = total_sono.values.tolist()

In [ ]:
criar_pieplot(dataframe=data, values=values, labels=labels)

In [ ]:
ordem = ["segunda-feira", "terça-feira", "quarta-feira", "quinta-feira", "sexta-feira",
         "sábado", "domingo"]

In [ ]:
data_semana = data.copy()

In [ ]:
data_semana["dia_semana"] = pd.Categorical(data_semana["dia_semana"], categories=ordem, ordered=True)

In [ ]:
medias = data_semana.groupby("dia_semana")["pontuacao"].mean().round(2)

In [ ]:
medias_df = medias.reset_index()
medias_df.columns = ["dia_semana", "pontuacao"]

In [ ]:
medias_df = medias_df.sort_values("pontuacao", ascending=True)

In [ ]:
medias_df

In [ ]:
plt.figure(figsize=(16, 8))
sns.barplot(data=medias_df, x="dia_semana", y="pontuacao", order=medias_df["dia_semana"])
plt.title("Média de Pontuação por dia da Semana", size=16)
plt.xlabel("Dia da Semana", size=14)
plt.ylabel("Pontuação", size=14)
plt.tight_layout()
plt.show()

In [ ]:
data_ordenado = data.sort_values(by="pontuacao", ascending=False)
maiores = data_ordenado.head(15)
menores = data_ordenado.tail(15)

In [ ]:
maiores_dias_semana = maiores["dia_semana"].value_counts()
maiores_dias_semana

In [ ]:
menores_dias_semana = menores["dia_semana"].value_counts()
menores_dias_semana

In [ ]:
maiores

In [ ]:
menores

In [ ]:
media_maiores = maiores[["pontuacao", "duracao", "sono_leve_perc", "sono_profundo_perc", "REM_perc", "tempo_acordado", "vezes_acordado"]].mean().round(2)
media_menores = menores[["pontuacao", "duracao", "sono_leve_perc", "sono_profundo_perc", "REM_perc", "tempo_acordado", "vezes_acordado"]].mean().round(2)

In [ ]:
imprimir_medias(media_maiores)

In [ ]:
imprimir_medias(media_menores)


In [ ]:
imprimir_diferenca_medias(media_maiores, media_menores)

In [ ]:
categorias = ["Pontuação", "Duração", "Sono Leve (%)", "Sono Profundo (%)", "REM (%)", "Tempo Acordado", "Vezes Acordado"]

In [ ]:
maiores_vals = media_maiores.reset_index(drop=True)
menores_vals = media_menores.reset_index(drop=True)

In [ ]:
criar_side_by_side(alto=maiores_vals, baixo=menores_vals, categorias=categorias, title="Média das 15 maiores e menores pontuações")

In [ ]:
maior_pontuacao = maiores.iloc[0].drop(["data", "dia_semana", "regularidade", "sono_leve", "sono_profundo", "REM"])
menor_pontuacao = menores.iloc[-1].drop(["data", "dia_semana", "regularidade","sono_leve", "sono_profundo", "REM"])

In [ ]:
maior_pontuacao

In [ ]:
criar_side_by_side(alto=maior_pontuacao, baixo=menor_pontuacao, categorias=categorias, title="Comparação da Maior e Menor Pontuação")